In [1]:
import sys
import rospy 
import actionlib
import threading
import time
import actionlib.msg
import jupyros as jr
import assignment_2_2023.msg
from nav_msgs.msg import Odometry
import ipywidgets as widgets
from IPython.display import display
from assignment_2_2023.msg import Info
from assignment_2_2023.msg import PlanningAction, PlanningGoal
from geometry_msgs.msg import Point, Pose, Twist, PoseStamped
from assignment_2_2023.srv import target, targetResponse

In [ ]:
#Initialization of elements
pose = PoseStamped()
target_reached = 0
target_canceled = 0

In [ ]:
# Initialize ROS node
rospy.init_node("node_a", anonymous=True)

In [ ]:
# Create actions client and connect to server
action_name = '/reaching_goal'
client = actionlib.SimpleActionClient(action_name, PlanningAction)
server_ok = client.wait_for_server(timeout=rospy.Duration(5.0))
if server_ok:
    rospy.loginfo(f'[{action_name.upper()}] Connection to server successful.')
else:
    rospy.logerr(f'[{action_name.upper()}] Unable to connect to server.')

In [ ]:
#Publish
pub_info = rospy.Publisher('/bot_info', Info, queue_size=1)

In [ ]:
# Function to handle goal result
def goal_done_cb(state, result):
    global target_reached, target_canceled
    with output:
        if state == actionlib.GoalStatus.SUCCEEDED:
            target_reached += 1
            goal_status.value = "Goal reached"
        else:
            target_canceled += 1
            goal_status.value = "Goal failed/canceled"
        vix_goals.update(target_reached, target_canceled)

# Function to handle goal feedback
def goal_feedback_cb(feedback):
    with output:
        pass


In [ ]:
# Widgets for input and output
x_input = widgets.FloatText(description='X Coordinate:')
y_input = widgets.FloatText(description='Y Coordinate:')
send_goal_button = widgets.Button(description='Send Goal')
stop_goal_button = widgets.Button(description='Stop Goal')
goal_status = widgets.Text(value="No goal set", description='Goal Status:', disabled=True)

# Output box for feedback
output = widgets.Output()

# Function to send a goal
def send_goal(button):
    with output:
        x_goal = x_input.value
        y_goal = y_input.value
        
        pose = PoseStamped()
        pose.pose.position.x = x_goal
        pose.pose.position.y = y_goal
        pose.pose.position.z = 0
        goal = PlanningGoal(target_pose=pose)

        client.send_goal(goal, done_cb=goal_done_cb, feedback_cb=goal_feedback_cb)
        goal_status.value = f"Goal set at ({x_goal}, {y_goal}, 0)"
        print(f"Goal sent to ({x_goal}, {y_goal})")
        
        
# Function to stop a goal
def stop_goal(button):
    with output:
        client.cancel_goal()
        goal_status.value = "Goal canceled"
        print("Goal canceled.")
        
        
send_goal_button.on_click(send_goal)
stop_goal_button.on_click(stop_goal)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import ipywidgets as widgets
import jupyros as jr
from nav_msgs.msg import Odometry 
%matplotlib notebook

class Visualiser:
    
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.ln, = plt.plot([], [], 'g^', markersize=3, linestyle='-', linewidth=1) 
        self.x_data, self.y_data = [] , []
    
    def plot_init(self):
        self.ax.set_xlim(20, -20)
        self.ax.set_ylim(20, -20)
        return self.ln
    
    def odom_callback(self, msg):
        self.y_data.append(msg.pose.pose.position.y)
        self.x_data.append(msg.pose.pose.position.x)
    
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

    def add_goal(self, x, y):
        self.ax.plot(x, y, marker="+",markeredgecolor="cyan")

In [ ]:
class VisualizerGoals:
    def __init__(self):
        self.fig, self.ax = plt.subplots()
        self.res = ['Successful', 'Failed']
        self.n_goals = [0, 0]
        self.bar_colors = ['tab:green', 'tab:red']
        self.bar = self.ax.bar(self.res, self.n_goals, color=self.bar_colors)
        self.ax.set_title('Counting the State of the Goals - Successful or Failed')
        self.ax.set_ylim(0, 10)  
        self.ax.set_yticks(range(0, 11))  

    def update(self, s, c):
        self.n_goals = [s, c]
        for bar, height in zip(self.bar, self.n_goals):
            bar.set_height(height)
        self.ax.relim()
        self.ax.autoscale_view()
        self.fig.canvas.draw()

In [ ]:
# Odometry callback function to publish robot information
def clbk_odom(msg):
    
    pub_info.publish(Info(
        x=msg.pose.pose.position.x,
        y=msg.pose.pose.position.y,
        vel_x=msg.twist.twist.linear.x,
        vel_y=msg.twist.twist.linear.y
    ))

In [ ]:
# Service callback to get information about goals
def get_info_goal(req):
    return targetResponse(target_reached, target_canceled)

In [ ]:
# Set up publishers, subscribers, and services
pub_info = rospy.Publisher("/bot_info", Info, queue_size=1)
rospy.Subscriber("/odom", Odometry, clbk_odom)
rospy.Service("goal_info", target, get_info_goal)

In [ ]:
# wait for the Action Server
client.wait_for_server()

In [ ]:
vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

In [ ]:
# Display layout with text inputs and buttons
layout = widgets.VBox([x_input, y_input, send_goal_button, stop_goal_button, goal_status, output])
display(layout)

In [ ]:
vix_goals = VisualizerGoals()